In [ ]:
# default_exp hpopt
# default_cls_lvl 3

In [ ]:
#hide
%load_ext line_profiler
%matplotlib notebook

# Hyperparameter Optimization Module
> Pytorch Models for Sequential Data

In [ ]:
#export
from seqdata.core import *
from seqdata.models.core import *
from seqdata.learner import *
from fastai2.basics import *
from fastai2.callback.schedule import *
from fastai2.callback.rnn import *
from fastai2.callback.tracker import *

import ray
from ray import tune
from ray.tune import Trainable
from ray.tune.schedulers import *
from ray.tune.trial import ExportFormat

In [ ]:
f_paths = '/mnt/data/Systemidentification/WienerHammerstein/'
hdf_files = L([f for f in get_hdf_files(f_paths) if 'test' not in str(f)])
tfm_src = CreateDict([DfHDFCreateWindows(win_sz=400,stp_sz=100,clm='u')])
dls = DataBlock(blocks=(SequenceBlock.from_hdf(['u'],TensorSequencesInput),
                        SequenceBlock.from_hdf(['y'],TensorSequencesOutput)),
                get_items=tfm_src,
                splitter=ApplyToDict(FuncSplitter(lambda o: 'valid' in str(o)))).dataloaders(hdf_files)

## optimizer core

First we need a log uniform distibution for variables with vast value ranges

In [ ]:
#export
def log_uniform(min_bound, max_bound, base=10):
    '''uniform sampling in an exponential range'''
    logmin = np.log(min_bound) / np.log(base)
    logmax = np.log(max_bound) / np.log(base)
    def _sample():
        return base**(np.random.uniform(logmin, logmax))
    return _sample

In [ ]:
[log_uniform(1e-8, 1e-2)() for _ in range(5)]

[0.0006705651619449839,
 3.033112015022537e-07,
 1.9762028258780075e-07,
 0.0014911616745581977,
 9.689375387920038e-06]

In [ ]:
#export
class LearnerTrainable(tune.Trainable):

    def _setup(self, config):
        self.create_lrn = ray.get(config['create_lrn'])
        self.dls = ray.get(config['dls'])

        self.lrn = self.create_lrn(self.dls,config)

    def _train(self):
        with self.lrn.no_bar(): self.lrn.fit(1)
        train_loss,valid_loss,rmse = self.lrn.recorder.values[-1]
        return {'train_loss': train_loss,
                'valid_loss': valid_loss,
                'mean_loss': rmse}

    def _save(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model.pth")
        torch.save(self.lrn.model.state_dict(), checkpoint_path)
        return checkpoint_path

    def _restore(self, checkpoint_path):
        self.lrn.model.load_state_dict(torch.load(checkpoint_path))

    def _export_model(self, export_formats, export_dir):
        if export_formats == [ExportFormat.MODEL]:
            path = os.path.join(export_dir, "exported_model")
            torch.save(self.lrn.model.state_dict(), path)
            return {ExportFormat.MODEL: path}
        else:
            raise ValueError("unexpected formats: " + str(export_formats))

    # the learner class will be recreated with every perturbation, saving the model
    # that way the new hyperparameter will be applied
    def reset_config(self, new_config):
        model_state = self.lrn.model.state_dict()
#         import pdb; pdb.set_trace()
        self.lrn = self.create_lrn(self.dls,new_config)
        
        #restore trainable parameters, keeping the new hyperparameters in the model like dropout
        self.lrn.model.load_state_dict(model_state)
        
        self.config = new_config
        return True

In [ ]:
#export
from fastai2.callback.tracker import SaveModelCallback 
def learner_optimize(config, checkpoint_dir=None):
        create_lrn = ray.get(config['create_lrn'])
        dls = ray.get(config['dls'])
        
        #Scheduling Parameters for training the Model
        lrn_kwargs = {'n_epoch':100,'pct_start':0.5}
        for attr in ['n_epoch','pct_start']:
            if attr in config: lrn_kwargs[attr] = config[attr]

        lrn = create_lrn(dls,config)
        lrn.lr = config['lr'] if 'lr' in config else 3e-3
        lrn.add_cb(CBRayReporter() if 'reporter' not in config else ray.get(config['reporter'])())
        lrn.add_cb(SaveModelCallback())
        with lrn.no_bar(): 
            ray.get(config['fit_method'])(lrn,**lrn_kwargs)

The mutation config dictionary consists of functions that sample from a distribution. In order to retrieve a dictionary with one realisation we need the function sample_config

In [ ]:
#export
def sample_config(config):
    ret_conf = config.copy()
    for k in ret_conf:
        ret_conf[k]=ret_conf[k]()
    return ret_conf

In [ ]:
#export
class CBRayReporter(Callback):
    "`Callback` reports progress after every epoch to the ray tune logger"

    def after_epoch(self):
        train_loss,valid_loss,rmse = self.learn.recorder.values[-1]
        tune.report(train_loss=train_loss,
                        valid_loss=valid_loss,
                        mean_loss=rmse)


In [ ]:
#export
class HPOptimizer():
    def __init__(self,create_lrn,dls):
        self.create_lrn = create_lrn
        self.dls = dls
        self.analysis = None
    
    @delegates(ray.init)
    def start_ray(self,**kwargs):
        ray.shutdown()
        ray.init(**kwargs)
        
    def stop_ray(self):
        ray.shutdown()
        
    
        

    @delegates(tune.run, keep=True)
    def optimize(self,config,optimize_func=learner_optimize,resources_per_trial={"gpu": 1.0},verbose=1,**kwargs):
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        if 'fit_method' not in config: config['fit_method'] = ray.put(Learner.fit_flat_cos)

        self.analysis = tune.run(
            optimize_func,
            config=config,
            resources_per_trial=resources_per_trial,
            verbose=verbose,
            **kwargs)
        return self.analysis
        
    @delegates(tune.run, keep=True)
    def optimize_pbt(self,opt_name,num_samples,config,mut_conf,freq=2,
                 stop={"training_iteration": 40 },
                 resources_per_trial={"gpu": 1 },
                 resample_probability=0.25,
                 quantile_fraction=0.25,
                 **kwargs):
        self.mut_conf = mut_conf
        
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        
        

        
        scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="mean_loss",
        mode="min",
        perturbation_interval=freq,
        resample_probability=resample_probability,
        quantile_fraction=quantile_fraction,
        hyperparam_mutations=mut_conf)
        
        self.analysis = tune.run(
            LearnerTrainable,
            name=opt_name,
            scheduler=scheduler,
            reuse_actors=True,
            verbose=1,
            stop=stop,
            checkpoint_score_attr="mean_loss",
            checkpoint_freq=freq,
            keep_checkpoints_num=4,
            num_samples=num_samples,
            resources_per_trial=resources_per_trial,
            config=config,
            **kwargs)
        return self.analysis
    
    def best_model(self):
        if self.analysis is None: raise Exception
        model = self.create_lrn(self.dls,sample_config(self.mut_conf)).model
        f_path = self.analysis.get_best_trial('mean_loss').checkpoint.value
        model.load_state_dict(torch.load(f_path))
        return model

### Test Population Based Training

In [ ]:
def create_lrn(dls,config):
    lr = config['lr']
    alpha = config['alpha']
    beta = config['beta']
    weight_p = config['weight_p']
    
    lrn = RNNLearner(dls,cbs=[TimeSeriesRegularizer(alpha,beta)],weight_p=weight_p)
    lrn.lr = lr
    return lrn

In [ ]:
config={
            "lr": tune.loguniform(1e-2, 1e-4),
            "alpha": tune.loguniform(1e-5, 10),
            "beta": tune.loguniform(1e-5, 10),
            "weight_p": tune.uniform(0, 0.5)}
mut_conf = {# distribution for resampling
            "lr": log_uniform(1e-8, 1e-2),
            "alpha": log_uniform(1e-5, 10),
            "beta": log_uniform(1e-5, 10),
            "weight_p": lambda: np.random.uniform(0, 0.5)}

hp_opt = HPOptimizer(create_lrn,dls)
hp_opt.start_ray()
hp_opt.optimize_pbt('pbt_test',4,config,mut_conf,freq=1,
                 stop={"training_iteration": 3 },
                 resources_per_trial={"gpu": 0.5},
                 local_dir='/mnt/data/ray_results')#no cpu count is necessary

Trial name,status,loc,alpha,beta,lr,weight_p,loss,iter,total time (s),train_loss,valid_loss,neg_mean_loss
LearnerTrainable_35897_00000,TERMINATED,,0.00277347,0.00580332,0.00189658,0.286271,0.245653,3,8.01477,0.0551403,0.060397,-0.245653
LearnerTrainable_35897_00001,TERMINATED,,0.552176,0.0318722,0.0010794,0.163314,0.245539,3,7.79972,0.055529,0.0603454,-0.245539
LearnerTrainable_35897_00002,TERMINATED,,1.59938,6.62838,0.000283733,0.133931,0.246883,3,8.05359,0.0572483,0.0610083,-0.246883
LearnerTrainable_35897_00003,TERMINATED,,0.00346684,0.0048361,0.00158048,0.357838,0.2447,3,7.89004,0.0548479,0.0599264,-0.2447


In [ ]:
hp_opt.best_model()

ValueError: No `mode` has been passed and  `default_mode` has not been set. Please specify the `mode` parameter.

### Test Grid Search

In [ ]:
def create_lrn(dls,config):
    lrn = RNNLearner(dls,hidden_size=config['hidden_size'],metrics=[fun_rmse,mse])
    return lrn

In [ ]:
class CustomReporter(Callback):
    "`Callback` reports progress after every epoch to the ray tune logger"

    def after_epoch(self):
        train_loss,valid_loss,rmse,mse = self.learn.recorder.values[-1]
        tune.report(train_loss=train_loss,
                        valid_loss=valid_loss,
                        mean_loss=rmse,
                        mse=mse)

In [ ]:
hp_opt = HPOptimizer(create_lrn,dls)

In [ ]:
hp_opt.start_ray(local_mode=False)

2020-10-04 00:46:42,948	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


In [ ]:
search_space = {
    "hidden_size": tune.grid_search([10,20,50,100]),
    'n_epoch':20,
    'reporter':ray.put(CustomReporter)
}

In [ ]:
hp_opt.optimize(resources_per_trial={"gpu": 0.5},
                config=search_space)

Trial name,status,loc,hidden_size,loss,iter,total time (s),train_loss,valid_loss,mse
learner_optimize_50b24_00000,RUNNING,130.149.48.236:58877,10,0.263788,3,10.8797,0.0728292,0.0696044,0.0696044
learner_optimize_50b24_00001,RUNNING,130.149.48.236:58886,20,0.246422,2,10.8895,0.0563075,0.060776,0.060776
learner_optimize_50b24_00002,RUNNING,130.149.48.236:58865,50,0.247049,2,10.0826,0.0629461,0.0610912,0.0610912
learner_optimize_50b24_00003,RUNNING,130.149.48.236:58843,100,0.24187,3,11.9689,0.0554109,0.0585442,0.0585442


(pid=58865) (#6) [2,0.06042690947651863,0.062047552317380905,0.24902784824371338,0.062047552317380905,'00:02']
(pid=58877) (#6) [3,0.06832592189311981,0.06365511566400528,0.25223809480667114,0.06365511566400528,'00:02']
(pid=58886) (#6) [2,0.05600298196077347,0.06078524515032768,0.24643062055110931,0.06078524515032768,'00:02']
(pid=58843) (#6) [3,0.0529298298060894,0.04839551821351051,0.21991665661334991,0.04839551821351051,'00:02']
(pid=58877) (#6) [4,0.06469633430242538,0.061240993440151215,0.24737101793289185,0.061240993440151215,'00:02']


KeyboardInterrupt: 

(pid=58865) (#6) [3,0.05882358178496361,0.060489363968372345,0.24586324393749237,0.060489363968372345,'00:02']
(pid=58886) (#6) [3,0.05570889264345169,0.060445960611104965,0.24575257301330566,0.060445960611104965,'00:02']
(pid=58843) (#6) [4,0.04667248576879501,0.02904750406742096,0.1703760176897049,0.02904750406742096,'00:02']


In [ ]:
hp_opt.analysis.get_best_config('mean_loss',mode='min')

### Test Random Search

In [ ]:
#hide
from nbdev.export import *
notebook2script()